In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

# Function to randomly sample data without replacement
def select_sample(X, y, sample_size=1000):
    indices = np.random.choice(X.shape[0], sample_size, replace=False)
    return X[indices], y[indices]

# Function to initialize a decision tree node
def initialize_node(value=None):
    return {'value': value, 'left': None, 'right': None, 'feature': None, 'threshold': None}

# Function to check if a node is a terminal/leaf node
def is_terminal_node(node):
    return node['value'] is not None

# Function to identify feature types (categorical/numerical)
def identify_feature_types(X):
    feature_types = []
    for i in range(X.shape[1]):
        unique_vals = np.unique(X[:, i])
        if isinstance(unique_vals[0], (int, float)) and len(unique_vals) > 10:
            feature_types.append("numerical")
        else:
            feature_types.append("categorical")
    return feature_types

# Function to split the dataset based on a feature and a threshold
def partition_data(X, feature, threshold):
    left_indices = np.where(X[:, feature] <= threshold)[0]
    right_indices = np.where(X[:, feature] > threshold)[0]
    return left_indices, right_indices

# Function to compute entropy for the target labels
def calculate_entropy(y):
    label_counts = Counter(y)
    total_samples = len(y)
    entropy_val = 0.0
    for count in label_counts.values():
        probability = count / total_samples
        if probability > 0:
            entropy_val -= probability * np.log2(probability)
    return entropy_val

# Function to compute information gain based on entropy
def compute_information_gain(y, left_y, right_y):
    total_samples = len(y)
    parent_entropy = calculate_entropy(y)
    left_entropy = calculate_entropy(left_y)
    right_entropy = calculate_entropy(right_y)
    weighted_entropy = (len(left_y) / total_samples) * left_entropy + (len(right_y) / total_samples) * right_entropy
    return parent_entropy - weighted_entropy

# Function to determine the best split for the data
def find_optimal_split(X, y, feature_types):
    best_gain = -np.inf
    optimal_feature = None
    optimal_threshold = None
    num_features = X.shape[1]

    for feature in range(num_features):
        if feature_types[feature] == "numerical":
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                left_indices, right_indices = partition_data(X, feature, threshold)
                if len(left_indices) == 0 or len(right_indices) == 0:
                    continue
                gain = compute_information_gain(y, y[left_indices], y[right_indices])
                if gain > best_gain:
                    best_gain = gain
                    optimal_feature = feature
                    optimal_threshold = threshold
        else:
            unique_vals = np.unique(X[:, feature])
            for value in unique_vals:
                left_indices, right_indices = partition_data(X, feature, value)
                if len(left_indices) == 0 or len(right_indices) == 0:
                    continue
                gain = compute_information_gain(y, y[left_indices], y[right_indices])
                if gain > best_gain:
                    best_gain = gain
                    optimal_feature = feature
                    optimal_threshold = value

    return optimal_feature, optimal_threshold

# Function to construct the decision tree
def construct_tree(X, y, feature_types):
    if len(np.unique(y)) == 1:
        return initialize_node(value=y[0])

    optimal_feature, optimal_threshold = find_optimal_split(X, y, feature_types)

    if optimal_feature is None:
        return initialize_node(value=Counter(y).most_common(1)[0][0])

    left_indices, right_indices = partition_data(X, optimal_feature, optimal_threshold)

    if len(left_indices) == 0 or len(right_indices) == 0:
        return initialize_node(value=Counter(y).most_common(1)[0][0])

    left_branch = construct_tree(X[left_indices], y[left_indices], feature_types)
    right_branch = construct_tree(X[right_indices], y[right_indices], feature_types)

    root_node = initialize_node()
    root_node['feature'] = optimal_feature
    root_node['threshold'] = optimal_threshold
    root_node['left'] = left_branch
    root_node['right'] = right_branch

    return root_node

# Function to make predictions using a decision tree
def generate_predictions(X, tree):
    predictions = np.zeros(X.shape[0])
    for i, sample in enumerate(X):
        current_node = tree
        while not is_terminal_node(current_node):
            if sample[current_node['feature']] <= current_node['threshold']:
                current_node = current_node['left']
            else:
                current_node = current_node['right']
        predictions[i] = current_node['value']
    return predictions

# Main function implementing bagging and bias-variance decomposition
def bagging_algorithm(X_train, y_train, X_test, y_test, iterations=10, num_trees=20, sample_size=300):
    single_tree_predictions = np.zeros((iterations, X_test.shape[0]))
    aggregated_bagging_predictions = np.zeros((iterations, X_test.shape[0]))

    feature_types = identify_feature_types(X_train)

    for i in range(iterations):
        # Select sample without replacement
        X_sample, y_sample = select_sample(X_train, y_train, sample_size)

        # Train a single decision tree
        single_tree_model = construct_tree(X_sample, y_sample, feature_types)
        single_tree_predictions[i] = generate_predictions(X_test, single_tree_model)

        # Train an ensemble of decision trees using bagging
        ensemble_predictions = np.zeros((num_trees, X_test.shape[0]))
        for t in range(num_trees):
            X_bagged_sample, y_bagged_sample = select_sample(X_train, y_train, sample_size)
            tree = construct_tree(X_bagged_sample, y_bagged_sample, feature_types)
            ensemble_predictions[t] = generate_predictions(X_test, tree)

        # Average predictions for bagged trees
        aggregated_bagging_predictions[i] = np.mean(ensemble_predictions, axis=0)

    # Compute bias and variance for single decision trees
    single_tree_bias, single_tree_variance = calculate_bias_variance(single_tree_predictions, y_test)

    # Compute bias and variance for bagged decision trees
    bagged_bias, bagged_variance = calculate_bias_variance(aggregated_bagging_predictions, y_test)

    return single_tree_bias, single_tree_variance, bagged_bias, bagged_variance

# Function to calculate bias and variance
def calculate_bias_variance(predicted_outputs, true_labels):
    average_prediction = np.mean(predicted_outputs, axis=0)
    bias_squared = np.mean((average_prediction - true_labels) ** 2)
    variance = np.mean(np.var(predicted_outputs, axis=0, ddof=1))
    return bias_squared, variance

# Data preprocessing: Convert categorical features to numeric
def transform_data(df):
    categorical_cols = df.select_dtypes(include=['object']).columns
    for column in categorical_cols:
        df[column] = pd.Categorical(df[column]).codes
    return df

# Main function for executing the workflow
def run():
    # Load the training and test datasets
    headers = ['age', 'job', 'marital', 'education', 'default', 'balance',
               'housing', 'loan', 'contact', 'day', 'month', 'duration',
               'campaign', 'pdays', 'previous', 'poutcome', 'label']

    train_df = pd.read_csv("train.csv", names=headers)
    test_df = pd.read_csv("test.csv", names=headers)

    # Transform the datasets by converting categorical to numeric features
    train_df = transform_data(train_df)
    test_df = transform_data(test_df)

    # Separate features and labels for training and testing
    X_train = train_df.drop('label', axis=1).values
    y_train = np.where(train_df['label'] == 1, 1, -1)
    X_test = test_df.drop('label', axis=1).values
    y_test = np.where(test_df['label'] == 1, 1, -1)

    # Perform bagging and bias-variance analysis
    single_tree_bias, single_tree_variance, bagged_bias, bagged_variance = bagging_algorithm(X_train, y_train, X_test, y_test)

    # Display the results
    print(f"Single Tree Bias^2: {single_tree_bias}, Variance: {single_tree_variance}, General Squared Error: {single_tree_bias + single_tree_variance}")
    print(f"Bagged Trees Bias^2: {bagged_bias}, Variance: {bagged_variance}, General Squared Error: {bagged_bias + bagged_variance}")


if __name__ == "__main__":
    run()


Single Tree Bias^2: 0.34012800000000004, Variance: 0.3541688888888889, General Squared Error: 0.694296888888889
Bagged Trees Bias^2: 0.29985185999999997, Variance: 0.017175044444444447, General Squared Error: 0.3170269044444444
